# Advanced NumPy

## `numpy` internals

In [1]:
import numpy as np
np.random.seed(2374)

In [2]:
arr = np.random.randint(10, size=(8,8))

In [3]:
arr.itemsize, arr.dtype

(8, dtype('int64'))

In [4]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [5]:
# How to step through array memory?

arr.strides

(64, 8)

In [6]:
arr.strides[0] == arr.shape[1] * arr.itemsize

True

In [7]:
# But what about views?

arr_view = arr[::2, 1:]

In [8]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [9]:
arr_view

array([[7, 6, 8, 2, 9, 2, 0],
       [6, 9, 3, 3, 5, 1, 2],
       [3, 9, 5, 2, 4, 5, 2],
       [8, 7, 4, 7, 3, 8, 2]])

In [10]:
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [11]:
arr_view.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [12]:
# View always has base array
arr_view.base is arr

True

In [13]:
arr_view.strides

(128, 8)

In [14]:
arr_view.strides[0] == arr_view.shape[1] * arr_view.itemsize

False

In [15]:
np.byte_bounds(arr_view)[0] - np.byte_bounds(arr)[0]

8

In [16]:
np.byte_bounds(arr_view)[1] - np.byte_bounds(arr)[1]

-64

In [17]:
arr.T.strides

(8, 64)

In [18]:
arr.T.base is arr

True

## Cache effects

In [19]:
large_arr = np.random.randint(100, size=(1000000,))

In [20]:
STEP = 8
larger_arr = np.random.randint(100, size=(1000000*STEP,))

In [21]:
%timeit -n 100 -r 3 large_arr.sum()

1.18 ms ± 99.5 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [22]:
%timeit -n 100 -r 3 larger_arr[::STEP].sum()

6.19 ms ± 399 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [23]:
del large_arr, larger_arr

In [24]:
large_arr = np.random.randint(100, size=(1000,1000))

In [25]:
%timeit -n 50 -r 3 large_arr.sum(axis=1)

944 µs ± 245 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [26]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=1)

1.11 ms ± 256 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [27]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=0)

1.22 ms ± 173 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [28]:
large_arr.T.base is large_arr

True

## Views and copies

In [29]:
%timeit -n 100 -r 3 large_arr_copy = large_arr.copy()

2.62 ms ± 41.4 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [30]:
%timeit -n 100 -r 3 large_arr + 1

3.14 ms ± 144 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [31]:
%timeit -n 100 -r 3 np.add(large_arr, 1)

3 ms ± 137 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [32]:
%timeit -n 100 -r 3 np.add(large_arr, 1, out=large_arr)

1.14 ms ± 53.4 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [33]:
np.add(large_arr, 1, out=large_arr)

array([[334, 316, 328, ..., 372, 315, 311],
       [371, 305, 342, ..., 317, 343, 372],
       [362, 320, 322, ..., 360, 313, 370],
       ...,
       [376, 360, 366, ..., 324, 342, 390],
       [356, 322, 360, ..., 339, 394, 301],
       [345, 333, 392, ..., 378, 395, 341]])

### Beware!

In [34]:
A = np.random.randint(10, size=(10,10))
B = np.random.randint(10, size=(10,))

In [35]:
A

array([[9, 9, 4, 7, 3, 9, 1, 6, 4, 5],
       [2, 5, 7, 6, 2, 4, 2, 7, 8, 6],
       [7, 0, 0, 8, 9, 2, 6, 3, 0, 2],
       [8, 3, 7, 4, 5, 0, 6, 7, 7, 4],
       [3, 2, 0, 5, 4, 2, 5, 8, 3, 9],
       [4, 4, 4, 2, 6, 0, 9, 0, 7, 3],
       [7, 2, 5, 7, 5, 1, 0, 7, 6, 9],
       [6, 9, 7, 6, 3, 9, 8, 7, 8, 6],
       [3, 0, 8, 4, 5, 5, 2, 1, 5, 9],
       [5, 2, 4, 9, 1, 9, 4, 8, 5, 0]])

In [36]:
B

array([5, 7, 6, 2, 5, 9, 7, 1, 9, 3])

In [37]:
A+B

array([[14, 16, 10,  9,  8, 18,  8,  7, 13,  8],
       [ 7, 12, 13,  8,  7, 13,  9,  8, 17,  9],
       [12,  7,  6, 10, 14, 11, 13,  4,  9,  5],
       [13, 10, 13,  6, 10,  9, 13,  8, 16,  7],
       [ 8,  9,  6,  7,  9, 11, 12,  9, 12, 12],
       [ 9, 11, 10,  4, 11,  9, 16,  1, 16,  6],
       [12,  9, 11,  9, 10, 10,  7,  8, 15, 12],
       [11, 16, 13,  8,  8, 18, 15,  8, 17,  9],
       [ 8,  7, 14,  6, 10, 14,  9,  2, 14, 12],
       [10,  9, 10, 11,  6, 18, 11,  9, 14,  3]])

In [38]:
np.add(A, B)

array([[14, 16, 10,  9,  8, 18,  8,  7, 13,  8],
       [ 7, 12, 13,  8,  7, 13,  9,  8, 17,  9],
       [12,  7,  6, 10, 14, 11, 13,  4,  9,  5],
       [13, 10, 13,  6, 10,  9, 13,  8, 16,  7],
       [ 8,  9,  6,  7,  9, 11, 12,  9, 12, 12],
       [ 9, 11, 10,  4, 11,  9, 16,  1, 16,  6],
       [12,  9, 11,  9, 10, 10,  7,  8, 15, 12],
       [11, 16, 13,  8,  8, 18, 15,  8, 17,  9],
       [ 8,  7, 14,  6, 10, 14,  9,  2, 14, 12],
       [10,  9, 10, 11,  6, 18, 11,  9, 14,  3]])

In [39]:
np.add(A, B, out=A)

array([[14, 16, 10,  9,  8, 18,  8,  7, 13,  8],
       [ 7, 12, 13,  8,  7, 13,  9,  8, 17,  9],
       [12,  7,  6, 10, 14, 11, 13,  4,  9,  5],
       [13, 10, 13,  6, 10,  9, 13,  8, 16,  7],
       [ 8,  9,  6,  7,  9, 11, 12,  9, 12, 12],
       [ 9, 11, 10,  4, 11,  9, 16,  1, 16,  6],
       [12,  9, 11,  9, 10, 10,  7,  8, 15, 12],
       [11, 16, 13,  8,  8, 18, 15,  8, 17,  9],
       [ 8,  7, 14,  6, 10, 14,  9,  2, 14, 12],
       [10,  9, 10, 11,  6, 18, 11,  9, 14,  3]])

In [40]:
np.add(A, B, out=B)

ValueError: non-broadcastable output operand with shape (10,) doesn't match the broadcast shape (10,10)

# Broadcasting

In [41]:
arr_2d = np.random.randint(10, size=(10, 3))
arr_1d_1 = np.random.randint(10, size=(3, ))
arr_1d_2 = np.random.randint(10, size=(10, ))

In [42]:
arr_2d

array([[9, 4, 0],
       [9, 1, 7],
       [6, 6, 8],
       [5, 3, 7],
       [6, 1, 2],
       [9, 1, 4],
       [1, 1, 4],
       [8, 3, 6],
       [9, 4, 6],
       [3, 3, 2]])

In [43]:
arr_1d_1

array([2, 6, 6])

In [44]:
arr_1d_2

array([0, 5, 8, 5, 4, 7, 1, 4, 4, 2])

In [45]:
(arr_2d + arr_1d_1) - arr_2d

array([[2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6],
       [2, 6, 6]])

In [46]:
arr_2d + arr_1d_1

array([[11, 10,  6],
       [11,  7, 13],
       [ 8, 12, 14],
       [ 7,  9, 13],
       [ 8,  7,  8],
       [11,  7, 10],
       [ 3,  7, 10],
       [10,  9, 12],
       [11, 10, 12],
       [ 5,  9,  8]])

In [47]:
arr_2d + arr_1d_2

ValueError: operands could not be broadcast together with shapes (10,3) (10,) 

In [48]:
(arr_2d + np.expand_dims(arr_1d_2, axis=1)) - arr_2d

array([[0, 0, 0],
       [5, 5, 5],
       [8, 8, 8],
       [5, 5, 5],
       [4, 4, 4],
       [7, 7, 7],
       [1, 1, 1],
       [4, 4, 4],
       [4, 4, 4],
       [2, 2, 2]])

In [49]:
arr_3d = np.random.randint(10, size=(7, 10, 3))

In [50]:
arr_1d_1.shape

(3,)

In [51]:
(arr_3d + arr_1d_1) - arr_3d

array([[[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]],

       [[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]],

       [[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]],

       [[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]],

       [[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]],

       [[2, 6, 6],
        [2, 6, 6],
  

In [52]:
arr_3d.shape, arr_1d_2.shape

((7, 10, 3), (10,))

In [53]:
arr_1d_2

array([0, 5, 8, 5, 4, 7, 1, 4, 4, 2])

In [54]:
(arr_3d + np.expand_dims(arr_1d_2, axis=1)) - arr_3d

array([[[0, 0, 0],
        [5, 5, 5],
        [8, 8, 8],
        [5, 5, 5],
        [4, 4, 4],
        [7, 7, 7],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4],
        [2, 2, 2]],

       [[0, 0, 0],
        [5, 5, 5],
        [8, 8, 8],
        [5, 5, 5],
        [4, 4, 4],
        [7, 7, 7],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4],
        [2, 2, 2]],

       [[0, 0, 0],
        [5, 5, 5],
        [8, 8, 8],
        [5, 5, 5],
        [4, 4, 4],
        [7, 7, 7],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4],
        [2, 2, 2]],

       [[0, 0, 0],
        [5, 5, 5],
        [8, 8, 8],
        [5, 5, 5],
        [4, 4, 4],
        [7, 7, 7],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4],
        [2, 2, 2]],

       [[0, 0, 0],
        [5, 5, 5],
        [8, 8, 8],
        [5, 5, 5],
        [4, 4, 4],
        [7, 7, 7],
        [1, 1, 1],
        [4, 4, 4],
        [4, 4, 4],
        [2, 2, 2]],

       [[0, 0, 0],
        [5, 5, 5],
  

Broadcasting rules:
    
- All input arrays with ndim smaller than the input array of largest ndim, have 1’s prepended to their shapes.
- The size in each dimension of the output shape is the maximum of all the input sizes in that dimension.
- An input can be used in the calculation if its size in a particular dimension either matches the output size in that dimension, or has value exactly 1.
- If an input has a dimension size of 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension. In other words, the stepping machinery of the ufunc will simply not step along that dimension (the stride will be 0 for that dimension).

## General rules

### Avoid loops

In [55]:
def square_loop(a):
    """Calculate square of an array in loop. We assume 1D array here."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i]*a[i]
    return result

In [56]:
large_arr = np.random.randint(100, size=(100000,))

In [57]:
%timeit -n 10 -r 3 square_loop(large_arr)

61.9 ms ± 1.74 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [58]:
%timeit -n 10 -r 3 np.square(large_arr)

372 µs ± 184 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


### Use broadcasting

In [59]:
def row_loop(a, b):
    """Calculate square of an array in loop. We assume 1D array here."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i] + b
    return result

In [60]:
large_arr = np.random.randint(100, size=(1000,1000))
large_b = np.random.randint(100, size=(1000,))

In [61]:
%timeit -n 10 -r 3 row_loop(large_arr, large_b)

6.62 ms ± 1.73 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [62]:
%timeit -n 10 -r 3 np.add(large_arr, large_b)

3.79 ms ± 670 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [63]:
%timeit -n 10 -r 3 np.add(large_arr, large_b, out=large_arr)

2.25 ms ± 420 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [64]:
np.add(large_arr, large_b)

array([[1047, 2801, 3071, ..., 1667, 2266,  909],
       [1115, 2730, 3058, ..., 1673, 2255,  959],
       [1036, 2734, 3088, ..., 1674, 2173,  926],
       ...,
       [1071, 2780, 3073, ..., 1676, 2172,  951],
       [1105, 2742, 3070, ..., 1620, 2191,  924],
       [1051, 2800, 3123, ..., 1706, 2176,  917]])

In [65]:
row_loop(large_arr, large_b)

array([[1047, 2801, 3071, ..., 1667, 2266,  909],
       [1115, 2730, 3058, ..., 1673, 2255,  959],
       [1036, 2734, 3088, ..., 1674, 2173,  926],
       ...,
       [1071, 2780, 3073, ..., 1676, 2172,  951],
       [1105, 2742, 3070, ..., 1620, 2191,  924],
       [1051, 2800, 3123, ..., 1706, 2176,  917]])

In [ ]:
np.expand_dims(np.arange(10), axis=0) + np.expand_dims(np.arange(10), axis=-1)

### Beware!

In [ ]:
A = np.random.randint(10, size=(10,10))
B = np.random.randint(10, size=(10,))

In [ ]:
A

In [ ]:
B

In [ ]:
A+B

In [ ]:
np.add(A, B)

In [ ]:
np.add(A, B, out=A)

In [ ]:
np.add(A, B, out=B)

# Linear algebra basics

In [ ]:
v = np.random.randint(10, size=(3,))
m = np.random.randint(10, size=(5, 3))

In [ ]:
m

In [ ]:
v

## Dot products, determinants, traces

In [ ]:
np.dot(m, v)

In [ ]:
np.dot(m, v.reshape((3,-1)))

In [ ]:
np.dot(v, m.T)

In [ ]:
s = np.random.randint(10, size=(3,3))
s_inv = np.linalg.inv(s)

In [ ]:
s

In [ ]:
s_inv

In [ ]:
np.dot(s_inv, s)

In [ ]:
np.linalg.det(s), np.linalg.det(s_inv)

In [ ]:
np.trace(s), np.trace(s_inv)

## Eigenvalues, eigenvectors

In [ ]:
evals, evectors = np.linalg.eig(s)

In [ ]:
evals.sum()

In [ ]:
s_diagonal = np.diag(evals)

In [ ]:
s_diagonal

And now our matrix can be decomposed as:
    
$$
s = VEV{-1}
$$

where $E$ is a diagonal matrix (with eigenvalues on main diagonal), and $V$ is a matrix where columns are eigenvectors.

In [ ]:
np.dot(evectors, np.dot(s_diagonal, np.linalg.inv(evectors)))

In [ ]:
s

In [2]:
X = np.array([[1,0,0],
              [1,0,0],
              [1,1,0],
              [1,1,0],
              [1,1,1]])
y = np.array([1,2,3,4,2])
Xt = 0.5 * np.array([[1,-1,0],
                     [-1,2,-1],
                     [0,-1,3]])
Xt*X*y

ValueError: operands could not be broadcast together with shapes (3,3) (5,3) 